In [20]:
!uv add letta-client --active
!uv add python-dotenv --active

Resolved 157 packages in 3ms
Audited 139 packages in 31ms
Resolved 157 packages in 2ms
Audited 139 packages in 1ms


In [ ]:
from letta_client import Letta
from dotenv import load_dotenv
import os

load_dotenv()

client = Letta(token=os.getenv("LETTA_API_KEY"))

In [24]:
def print_message(message):
    if message.message_type == "reasoning_message":
        print("🧠 Reasoning: " + message.reasoning)
    elif message.message_type == "assistant_message":
        print("🤖 Agent: " + message.content)
    elif message.message_type == "tool_call_message":
        print("🔧 Tool Call: " + message.tool_call.name + "\n" + message.tool_call.arguments)

    elif message.message_type == "tool_return_message":
        print("🔧 Tool Return: " + message.tool_return)
    elif message.message_type == "user_message":
        print("👤 User Message: " + message.content)

In [25]:
agent_state = client.agents.create(
    name="simple_agent",
    memory_blocks=[
        {
          "label": "human",
          "value": "My name is Charles",
          "limit": 10000 # character limit
        },
        {
          "label": "persona",
          "value": "You are a helpful assistant and you always use emojis"
        }
    ],
    model="openai/gpt-4o-mini",
    embedding="openai/text-embedding-3-small"
)

## Messaging an Agent

In [27]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "Hi, My name is Dani"
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

🔧 Tool Call: memory_replace
{"label": "human", "old_str": "My name is Charles", "new_str": "My name is Dani"}
🔧 Tool Return: The core memory block with label `human` has been edited. Review the changes and make sure they are as expected (correct indentation, no duplicate lines, etc). Edit the memory block again if necessary.
🤖 Agent: Got it, Dani! 😊 How's your day going?


## Inside Agent State

In [37]:
print(agent_state.system)

<base_instructions>
You are a helpful self-improving agent with advanced memory and file system capabilities.
<memory>
You have an advanced memory system that enables you to remember past interactions and continuously improve your own capabilities.
Your memory consists of memory blocks and external memory:
- Memory Blocks: Stored as memory blocks, each containing a label (title), description (explaining how this block should influence your behavior), and value (the actual content). Memory blocks have size limits. Memory blocks are embedded within your system instructions and remain constantly available in-context.
- External memory: Additional memory storage that is accessible and that you can bring into context with tools when needed.
Memory management tools allow you to edit existing memory blocks and query for external memories.
</memory>
<file_system>
You have access to a structured file system that mirrors real-world directory structures. Each directory can contain multiple files.

In [38]:
[t.name for t in agent_state.tools]

['conversation_search', 'memory_replace', 'memory_insert']

## Memory Block

In [33]:
agent_state.memory

Memory(agent_type='letta_v1_agent', blocks=[Block(value='My name is Charles', limit=10000, project_id=None, name=None, is_template=False, base_template_id=None, deployment_id=None, entity_id=None, preserve_on_migration=False, label='human', read_only=False, description='The human block: Stores key details about the person you are conversing with, allowing for more personalized and friend-like conversation.', metadata={}, hidden=None, id='block-8e30dc64-09b9-42c5-b638-fd5a1986c32e', created_by_id=None, last_updated_by_id=None, template_name=None, template_id=None), Block(value='You are a helpful assistant and you always use emojis', limit=20000, project_id=None, name=None, is_template=False, base_template_id=None, deployment_id=None, entity_id=None, preserve_on_migration=False, label='persona', read_only=False, description='The persona block: Stores details about your current persona, guiding how you behave and respond. This helps you to maintain consistency and personality in your inte

In [35]:
for message in client.agents.messages.list(agent_id=agent_state.id):
    print_message(message)

👤 User Message: hows it going????
🤖 Agent: Hey Charles! 😊 I'm doing great, thanks for asking! How about you? What's new with you today?
👤 User Message: Hi, My name is Dani
🔧 Tool Call: memory_replace
{"label": "human", "old_str": "My name is Charles", "new_str": "My name is Dani"}
🔧 Tool Return: The core memory block with label `human` has been edited. Review the changes and make sure they are as expected (correct indentation, no duplicate lines, etc). Edit the memory block again if necessary.
🤖 Agent: Got it, Dani! 😊 How's your day going?


### Passages Archival Memory

In [ ]:
passages = client.agents.passages.list(
    agent_id=agent_state.id,
)
passages

[]

## Section 2: Understanding core memory

In [39]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "my name actually Sarah "
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

🔧 Tool Call: memory_replace
{"label": "human", "old_str": "My name is Dani", "new_str": "My name is Sarah"}
🔧 Tool Return: The core memory block with label `human` has been edited. Review the changes and make sure they are as expected (correct indentation, no duplicate lines, etc). Edit the memory block again if necessary.
🤖 Agent: Thanks for clarifying, Sarah! 😊 How can I assist you today?


In [ ]:
print(response.usage.completion_tokens)
print(response.usage.prompt_tokens)
print(response.usage.step_count)

46
3916
2


### Retrieving new values

In [41]:
client.agents.blocks.retrieve(
    agent_id=agent_state.id,
    block_label="human"
).value

'My name is Sarah'

## Archival Memory

In [42]:
passages = client.agents.passages.list(
    agent_id=agent_state.id,
)
passages

[]

In [43]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "Save the information that 'bob loves cats' to archival"
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

🔧 Tool Call: memory_insert
{"label": "archival", "new_str": "Bob loves cats", "insert_line": -1}
🔧 Tool Return: 'Block field archival does not exist (available sections = human, persona)'
🤖 Agent: It seems that I don't have a specific archival section to save that information in. However, I can remember it for our ongoing conversation. Would you like me to do that? 😊


In [44]:
passages = client.agents.passages.list(
    agent_id=agent_state.id,
)
[passage.text for passage in passages]

[]

## Explicitly creating archival memories

In [45]:
client.agents.passages.create(agent_id=agent_state.id,text="Bob's loves boston terries")

[Passage(created_by_id='user-5bbacfce-21be-4e67-96ba-9b5e736afded', last_updated_by_id='user-5bbacfce-21be-4e67-96ba-9b5e736afded', created_at=datetime.datetime(2025, 11, 15, 3, 3, 2, 596888, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 11, 15, 3, 3, 2, 616915, tzinfo=datetime.timezone.utc), is_deleted=False, archive_id='archive-4516aab5-2bdd-4021-b9e6-79840390a047', source_id=None, file_id=None, file_name=None, metadata={}, tags=None, id='passage-cdee9063-e321-42fa-8857-924c7a8297e3', text="Bob's loves boston terries", embedding=[0.04421710595488548, -0.028221456333994865, 0.009562047198414803, 0.010223078541457653, -0.03515901416540146, -0.018600504845380783, -0.002061632461845875, 0.06356372684240341, -0.02158496342599392, -0.05544809252023697, -0.00271284650079906, -0.014333253726363182, 0.029190097004175186, -0.007290979381650686, -0.015236444771289825, -0.027305174618959427, 0.01476521510630846, 0.006067089736461639, 0.01807691715657711, 0.006950646173208952,

In [46]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "What animals do I like? Search archival."
        }
    ]
)

for message in response.messages:
    print_message(message)

🔧 Tool Call: conversation_search
{"query": "animals", "roles": ["user"], "limit": 5, "start_date": "2024-01-01", "end_date": "2025-11-15"}
🔧 Tool Return: Showing 4 results: [
  {
    "timestamp": "2025-11-15T03:01:05.858951+00:00",
    "time_ago": "2m ago",
    "role": "user",
    "relevance": {
      "rrf_score": 0.00819672131147541,
      "vector_rank": 1
    },
    "content": "Save the information that 'bob loves cats' to archival"
  },
  {
    "timestamp": "2025-11-15T02:25:56.792139+00:00",
    "time_ago": "37m ago",
    "role": "user",
    "relevance": {
      "rrf_score": 0.008064516129032258,
      "vector_rank": 2
    },
    "content": "hows it going????"
  },
  {
    "timestamp": "2025-11-15T02:57:52.379728+00:00",
    "time_ago": "5m ago",
    "role": "user",
    "relevance": {
      "rrf_score": 0.007936507936507936,
      "vector_rank": 3
    },
    "content": "my name actually Sarah "
  },
  {
    "timestamp": "2025-11-15T02:26:46.849081+00:00",
    "time_ago": "37m ago",

## Memory Block

In [53]:
blocks = client.agents.blocks.list(
    agent_id=agent_state.id,
)

blocks

[Block(value='You are a helpful assistant and you always use emojis', limit=20000, project_id=None, name=None, is_template=False, base_template_id=None, deployment_id=None, entity_id=None, preserve_on_migration=False, label='persona', read_only=False, description='The persona block: Stores details about your current persona, guiding how you behave and respond. This helps you to maintain consistency and personality in your interactions.', metadata={}, hidden=None, id='block-f92f9357-d456-4b95-a48f-08af14fe4400', created_by_id=None, last_updated_by_id=None, template_name=None, template_id=None),
 Block(value='My name is Sarah', limit=10000, project_id=None, name=None, is_template=False, base_template_id=None, deployment_id=None, entity_id=None, preserve_on_migration=False, label='human', read_only=False, description='The human block: Stores key details about the person you are conversing with, allowing for more personalized and friend-like conversation.', metadata={}, hidden=None, id='bl

In [58]:
human_block_id=None
for block in blocks:
    if block.label == "human":
        human_block_id = block.id

print(human_block_id)

block-8e30dc64-09b9-42c5-b638-fd5a1986c32e


### Accessing block prompt template

In [59]:
client.agents.core_memory.retrieve(
    agent_id=agent_state.id
).prompt_template

''

## Task Queue Memory

In [108]:
def task_queue_push(agent_state: "AgentState", task_description: str):
    """
    Push to a task queue stored in core memory.

    Args:
        task_description (str): A description of the next task you must accomplish.

    Returns:
        Optional[str]: None is always returned as this function
        does not produce a response.
    """

    from letta_client import Letta
    import os
    import json
    from dotenv import load_dotenv
    load_dotenv()

    client = Letta(token="sk-let-NzU1ZTEwN2MtMmUxNy00NGJkLWIxM2YtMjU0ZTlkNmVjZmRlOmRhYTE4MzM5LWQxYjUtNGExOC04NWFiLTcwMmVhOTUwZDEwNg==")

    block = client.agents.blocks.retrieve(
        agent_id=agent_state.id,
        block_label="tasks",
    )
    tasks = json.loads(block.value)
    tasks.append(task_description)

    # update the block value
    client.agents.blocks.modify(
        agent_id=agent_state.id,
        value=json.dumps(tasks),
        block_label="tasks"
    )
    return None

In [109]:
def task_queue_pop(agent_state: "AgentState"):
    """
    Get the next task from the task queue

    Returns:
        Optional[str]: Remaining tasks in the queue
    """

    from letta_client import Letta
    import json
    import os
    from dotenv import load_dotenv
    load_dotenv()

    client = Letta(token="sk-let-NzU1ZTEwN2MtMmUxNy00NGJkLWIxM2YtMjU0ZTlkNmVjZmRlOmRhYTE4MzM5LWQxYjUtNGExOC04NWFiLTcwMmVhOTUwZDEwNg==")

    # get the block
    block = client.agents.blocks.retrieve(
        agent_id=agent_state.id,
        block_label="tasks",
    )
    tasks = json.loads(block.value)
    if len(tasks) == 0:
        return None
    task = tasks[0]

    # update the block value
    remaining_tasks = json.dumps(tasks[1:])
    client.agents.blocks.modify(
        agent_id=agent_state.id,
        value=remaining_tasks,
        block_label="tasks"
    )
    return f"Remaining tasks {remaining_tasks}"


In [110]:
task_queue_pop_tool = client.tools.upsert_from_function(
    func=task_queue_pop

)
task_queue_push_tool = client.tools.upsert_from_function(
    func=task_queue_push
)

In [111]:
import json

task_agent = client.agents.create(
    # system=open("task_queue_system_prompt.txt", "r").read(),
    memory_blocks=[
        {
          "label": "tasks",
          "value": json.dumps([])
        }
    ],
    model="openai/gpt-4o-mini",
    embedding="openai/text-embedding-3-small",
    tool_ids=[task_queue_pop_tool.id, task_queue_push_tool.id],
    include_base_tools=False,
    tools=["send_message"]
)

In [112]:
[tool.name for tool in task_agent.tools]

['send_message', 'task_queue_push', 'task_queue_pop']

In [113]:
client.agents.blocks.retrieve(task_agent.id, block_label="tasks").value

'[]'

In [114]:
response_stream = client.agents.messages.create_stream(
    agent_id=task_agent.id,
    messages=[
        {
            "role": "user",
            "content": "Add 'start calling me Charles' and "
            + "'tell me a haiku about my name' as two seperate tasks."
        }
    ]
)

for chunk in response_stream:
    print_message(chunk)

🔧 Tool Call: task_queue_push
{"task_description": "Start calling me Charles."}
🔧 Tool Return: None
🔧 Tool Call: task_queue_push
{"task_description": "Tell me a haiku about my name."}
🔧 Tool Return: None
🤖 Agent: I've added the tasks: "Start calling me Charles" and "Tell me a haiku about your name." If there's anything else you'd like to do, just let me know!
